# Programación declarativa @ URJC
# Programación funcional
## Curso 21-22, convocatoria ordinaria (29 de octubre de 2021)
## Campus de Móstoles


# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

In [ ]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    def void[A]: Tree[A] = Empty()
    def leaf[A](a: A): Node[A] = Node(Empty(), a, Empty())
    def right[A](a: A, tree: Tree[A]): Node[A] = Node(Empty(), a, tree)
    def left[A](tree: Tree[A], a: A): Node[A] = Node(tree, a, Empty())
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

In [ ]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
        def reverse: List[A]
    }
}

In [ ]:
type Not[P] = P => Nothing

# Ejercicio 1
__(2 puntos)__


__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar los siguientes teoremas de la lógica intuicionista: 

$⊢ (p \wedge q) \rightarrow \neg(\neg p \vee \neg q) $

In [ ]:
def proof[P, Q]: (P, Q) => Not[Either[Not[P], Not[Q]]] = 
    { case (p: P, q: Q) => 
        {
            case Left(notP: (P => Nothing)) => notP(p)
            case Right(notQ: (Q => Nothing)) => notQ(q)
        }
    }

$⊢ (p \vee q) \rightarrow \neg(\neg p \wedge \neg q) $

In [ ]:
def proof[P, Q]: Either[P, Q] => Not[(Not[P], Not[Q])] = 
    { 
        case Left(p: P) => 
            { case (notP, notQ) => notP(p) }
        case Right(q: Q) => 
            { case (notP, notQ) => notQ(q) }
    }

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ \neg (\neg p \wedge \neg q) \rightarrow p \vee q $

Supóngase para ello que la ley del tercio excluso se cumple para las variables proposicionales $p$ y $q$, es decir, que las fórmulas  $p \vee \neg p$ y $q \vee \neg q$ pueden utilizarse como premisas.

In [ ]:
def proof[P, Q](middleP: Either[P, Not[P]], middleQ: Either[Q, Not[Q]]): Not[(Not[P], Not[Q])] => Either[P, Q] =  
    (middleP, middleQ) match {
        case (Left(p), _) => 
            _ => Left(p)
        case (_, Left(q)) => 
            _ => Right(q)
        case (Right(notP), Right(notQ)) => 
            _((notP, notQ))
    }

# Ejercicio 2 
__(2 puntos)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$, $Y$ y $Z$: 

$(1+Y+Z+Y*Z)^X \cong (Y+1)^X * (Z+1)^X$

A continuación se muestran unos casos de prueba de este isomorfismo para $X=Int$, $Y=Unit$ y $Z=Unit$:

In [ ]:
class IsoTest(
    from: (Int => Either[Unit, Either[Unit, Either[Unit, (Unit, Unit)]]]) => (Int => Option[Unit], Int => Option[Unit]), 
    to: ((Int => Option[Unit], Int => Option[Unit])) => Int => Either[Unit, Either[Unit, Either[Unit, (Unit, Unit)]]]
) extends FlatSpec with Matchers{
    
    val f: Int => Either[Unit, Either[Unit, Either[Unit, (Unit, Unit)]]] = 
        i => if (i % 4 == 0) Left(()) 
             else if (i % 4 == 1) Right(Left(()))
             else if (i % 4 == 2) Right(Right(Left(())))
             else Right(Right(Right(((),()))))
    
    val g: (Int => Option[Unit], Int => Option[Unit]) = 
        (i => if (i % 2 == 0) None else Some(()),
         i => if (i % 2 == 0) None else Some(()))
    
    "from-to" should "work" in {
        from(to(g))._1(0) shouldBe g._1(0)
        from(to(g))._1(1) shouldBe g._1(1)
        from(to(g))._1(2) shouldBe g._1(2)
        from(to(g))._1(3) shouldBe g._1(3)
        from(to(g))._2(0) shouldBe g._2(0)
        from(to(g))._2(1) shouldBe g._2(1)
        from(to(g))._2(2) shouldBe g._2(2)
        from(to(g))._2(3) shouldBe g._2(3)
    }
    
    "to-from" should "work" in {
        to(from(f))(0) shouldBe f(0)
        to(from(f))(1) shouldBe f(1)
        to(from(f))(2) shouldBe f(2)
        to(from(f))(3) shouldBe f(3)
    }
}

In [ ]:
def from[X, Y, Z](l: X => Either[Unit, Either[Y, Either[Z, (Y, Z)]]]): (X => Option[Y], X => Option[Z]) = 
    (x => l(x) match {
        case Right(Right(Right((y, _)))) => Some(y)
        case Right(Left(y)) => Some(y)
        case _ => None
    }, x => l(x) match {
        case Right(Right(Right((_, z)))) => Some(z)
        case Right(Right(Left(z))) => Some(z)
        case _ => None
    })

In [ ]:
def to[X, Y, Z](t: (X => Option[Y], X => Option[Z])): X => Either[Unit, Either[Y, Either[Z, (Y, Z)]]] = 
    t match {
        case (fy, fz) => 
            x => (fy(x), fz(x)) match {
                case (None, None) => Left(None)
                case (Some(y), None) => Right(Left(y))
                case (None, Some(z)) => Right(Right(Left(z)))
                case (Some(y), Some(z)) => Right(Right(Right((y,z))))
            }
    }

In [ ]:
run(new IsoTest(from[Int, Unit, Unit], to[Int, Unit, Unit]))

# Ejercicio 3
__(3 puntos)__

La función `lift` recibe una lista y devuelve una función que indexa sus elementos de acuerdo a su posición en la lista, es decir, si la función recibe un entero que representa una posición de la lista, devuelve el elemento almacenado en ella; en caso contrario, devuelve un valor que indica que en dicha posición no existe ningún elemento. A continuación se muestran distintos casos de prueba particularizados para una lista de caracteres:

In [ ]:
class TestLift(
    lift: List[Char] => Int => Option[Char]
) extends FlatSpec with Matchers{
    "lift" should "work" in {
        lift(List())(0) shouldBe None
        lift(List())(1) shouldBe None
        lift(List())(2) shouldBe None
        
        lift(List('a'))(0) shouldBe Some('a')
        lift(List('a'))(1) shouldBe None
        lift(List('a'))(2) shouldBe None
        
        lift(List('a','b'))(0) shouldBe Some('a')
        lift(List('a','b'))(1) shouldBe Some('b')
        lift(List('a','b'))(2) shouldBe None
        
        lift(List('a','b','c'))(0) shouldBe Some('a')
        lift(List('a','b','c'))(1) shouldBe Some('b')
        lift(List('a','b','c'))(2) shouldBe Some('c')
        lift(List('a','b','c'))(3) shouldBe None
    }
}

__a) (1 punto)__ Implementa la función `lift` de manera recursiva.

In [ ]:
def lift[A](l: List[A]): Int => Option[A] = 
    idx => l match {
        case Nil => 
            None
        case head :: tail => 
            if (idx == 0) Some(head)
            else lift(tail)(idx-1)
    }

In [ ]:
// Otra versión también aceptada como válida
def lift[A](l: List[A]): Int => Option[A] = 
    l match {
        case Nil => 
            _ => None
        case head :: tail => 
            idx => if (idx == 0) Some(head)
                   else lift(tail)(idx-1)
    }

In [ ]:
run(new TestLift(lift))

__b) (1 punto)__ Implementa la función `lift` utilizando la función de orden superior `foldRight`.

In [ ]:
def lift[A](l: List[A]): Int => Option[A] = 
    l.foldRight((_: Int) => None: Option[A]){
        (head, liftTail) => 
            idx => if (idx == 0) Some(head)
                   else liftTail(idx-1)
    }

In [ ]:
// Otra versión también aceptada como válida
def lift[A](l: List[A]): Int => Option[A] = 
    idx => l.foldRight((None: Option[A], l.length-1)){
        case (head, (None, `idx`)) => (Some(head), idx -1)
        case (_, (out, pos)) => (out, pos-1)
    }._1

In [ ]:
run(new TestLift(lift))

__c) (1 punto)__ Implementa la función `lift` mediante la función de orden superior `foldLeft`.

In [ ]:
def lift[A](l: List[A]): Int => Option[A] = 
    l.foldLeft((0, (_: Int) => None: Option[A])){
        case ((next, out), e) => 
            (next + 1, idx => 
                 if (idx == next) Some(e) 
                 else out(idx))
    }._2

In [ ]:
// Otra versión también aceptada como válida
def lift[A](l: List[A]): Int => Option[A] = 
    idx => l.foldLeft((0, None: Option[A])){
        case ((next, out), e) => 
            (next + 1, 
                 if (idx == next) Some(e) 
                 else out)
    }._2

In [ ]:
run(new TestLift(lift))

# Ejercicio 4
__(3 puntos)__

Una rama de un árbol binario consiste en una secuencia de nodos desde la raíz a una de sus hojas. Dado un árbol binario de números enteros, el coste de una rama consiste en la suma de todos los nodos que la componen. Se desea implementar una función que devuelva la rama del árbol de coste máximo. En caso de que existan varias ramas con el mismo coste máximo, la función deberá devolver una cualquiera de ellas. 

In [ ]:
class TestHighestCostBranch(highest: Tree[Int] => List[Int]) extends FlatSpec with Matchers{
    "highest path" should "work" in {
        highest(void) shouldBe List()
        highest(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe List(5,4,3,2,1)
        highest(node(left(leaf(8), 7), 0, node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe List(0, 7, 8)
        highest(node(left(leaf(8), 7), 0, node(leaf(6), 9, right(2, right(2, leaf(1)))))) should
            (be(List(0, 7, 8)) or be(List(0, 9, 6)))
    }
}

__a) (1,5 puntos)__ Implementa la función `highestCostBranch` recursivamente. La implementación deberá utilizar la siguiente función `sum` que calcula la suma de todos los elementos de una lista de enteros:

In [ ]:
def sum(list: List[Int]): Int = 
    list.foldRight(0)(_ + _)

In [ ]:
def highestCostBranch(tree: Tree[Int]): List[Int] = 
    tree match {
        case Empty() => List()
        case Node(left, root, right) => 
            val highestLeft: List[Int] = highestCostBranch(left)
            val highestRight: List[Int] = highestCostBranch(right)
            root :: (if (sum(highestRight) > sum(highestLeft)) highestRight else highestLeft)
    }

In [ ]:
run(new TestHighestCostBranch(highestCostBranch))

__b) (1,5 puntos)__ Implementa la función `highestCostBranch` mediante la función de orden superior `foldTree`, __sin__ utilizar la función auxiliar `sum`.

In [ ]:
def highestPath(tree: Tree[Int]): List[Int] = 
    foldTree(tree)((0, List[Int]())){
        case ((sumL, highestLeft), root, (sumR, highestRight)) => 
            if (sumL > sumR) (sumL + root, root :: highestLeft) 
            else (sumR + root, root :: highestRight)
    }._2

In [ ]:
run(new TestHighestCostBranch(highestCostBranch))